In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, LabelEncoder, LabelBinarizer, MaxAbsScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import roc_curve, auc, f1_score, recall_score, precision_score


In [2]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop([ 'Name', 'Ticket', 'Cabin'], inplace=True, axis=1)

In [8]:
ID = test['PassengerId']

In [9]:
test.drop('PassengerId', inplace=True, axis=1)

In [10]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [116]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_dum)
test_scaled = scaler.transform(test_dum)

In [118]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
test_pca = pca.transform(test_scaled)

In [113]:
pca.explained_variance_ratio_.cumsum()

array([0.93563708, 0.99919511])

In [115]:


X_pca

array([[-25.14767158,  -6.68317665],
       [ 39.3109204 ,   7.5771813 ],
       [-24.36369385,  -2.70551476],
       ...,
       [ -8.78450694,  -1.14992531],
       [ -2.28819413,  -3.28011753],
       [-24.37825518,   3.30280744]])

In [12]:
cat_features = ['Pclass', 'SibSp', 'Parch', 'Embarked', 'Sex']
num_features = ['Age', 'Fare']

In [14]:
for col in cat_features:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [15]:
train['Survived'] = train['Survived'].astype('category')

In [16]:
X_train = train.drop('Survived', axis=1)#.values
y_train = train['Survived']#.values 

In [17]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [18]:
y_train = y_train.astype('category')

In [38]:
test['Age'].fillna(29, inplace=True)
test['Fare'].fillna(35.6, inplace=True)
test_dum = pd.get_dummies(test)


In [30]:
X_train['Age'].fillna(28, inplace=True)
X_train_dum = pd.get_dummies(X_train)
#y_train_dum = pd.get_dummies(y_train)


In [36]:
test.Fare.mean()#isna().sum()

35.6271884892086

In [44]:
test_dum.drop('Parch_9', axis=1, inplace=True)

In [42]:
X_train_dum.columns

Index(['Age', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female',
       'Sex_male', 'SibSp_0', 'SibSp_1', 'SibSp_2', 'SibSp_3', 'SibSp_4',
       'SibSp_5', 'SibSp_8', 'Parch_0', 'Parch_1', 'Parch_2', 'Parch_3',
       'Parch_4', 'Parch_5', 'Parch_6', 'Embarked_C', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

In [40]:
X_train_dum.isna().sum().sum(), test_dum.isna().sum().sum()

(0, 0)

In [45]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_dum)
test_scaled = scaler.transform(test_dum)

In [109]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

In [110]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [111]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)])

In [83]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [381]:
BaggingClassifier().get_params().keys()

dict_keys(['base_estimator', 'bootstrap', 'bootstrap_features', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [146]:
steps = [ ('model', BaggingClassifier(base_estimator=SVC(), oob_score=True, verbose=True, n_estimators=100, max_samples=0.8, bootstrap=True, n_jobs=-1))]

steps = [ ('model', BaggingClassifier(estimators=[('xgbrf', XGBRFClassifier()), ('xgb', XGBClassifier()), ('gb', GradientBoostingClassifier()), ('rfc', RandomForestClassifier()), ('svc', SVC()), ('logreg', LogisticRegression())], voting='soft', verbose=True,)  )]

In [147]:
pipe = Pipeline(steps=steps)

In [149]:
pipe.score(X_pca, y_train)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished


0.7598204264870931

In [148]:
pipe.fit(X_pca, y_train)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


Pipeline(steps=[('model',
                 BaggingClassifier(base_estimator=SVC(), max_samples=0.8,
                                   n_estimators=100, n_jobs=-1, oob_score=True,
                                   verbose=True))])

In [150]:
pipe.fit(X_scaled, y_train)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


Pipeline(steps=[('model',
                 BaggingClassifier(base_estimator=SVC(), max_samples=0.8,
                                   n_estimators=100, n_jobs=-1, oob_score=True,
                                   verbose=True))])

In [151]:
pipe.score(X_scaled, y_train)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


0.8372615039281706

In [152]:
cross_val_score(pipe, X_scaled, y_train, cv=5).mean()

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Don

0.8114305442219572

In [122]:
xgb = XGBClassifier()
xgb.fit(X_pca, y_train)
xgb.score(X_pca, y_train)

[06:59:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9528619528619529

In [123]:
cross_val_score(xgb, X_scaled, y_train, cv=5)

[06:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

array([0.77094972, 0.81460674, 0.83707865, 0.76966292, 0.84269663])

In [327]:
import xgboost as xgb
from sklearn.linear_model import Perceptron

from sklearn.multiclass import OneVsRestClassifier

In [69]:
parameters = {
 'criterion': ['gini'],
 'max_depth': [1,2,5,10],
 'n_estimators': [100],
 'n_jobs': [-1],
 'oob_score': [True],
 'verbose': [1],
 'warm_start': [True]}

In [124]:
clf = RandomForestClassifier()
clf.fit(X_pca, y_train)

RandomForestClassifier()

In [126]:
clf.score(X_scaled, y_train.values)

0.9797979797979798

In [127]:
cross_val_score(clf, X_scaled, y_train, cv=5)

array([0.77094972, 0.79213483, 0.84831461, 0.76966292, 0.81460674])

In [91]:
pipe_cv = GridSearchCV(pipe, param_grid=parameters, cv=5, n_jobs=-1)

In [92]:
pipe_cv.fit(X_scaled, y_train)

ValueError: Invalid parameter criterion for estimator Pipeline(steps=[('model',
                 VotingClassifier(estimators=[('xgbrf',
                                               XGBRFClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bytree=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                        

In [75]:
pipe_cv.best_score_

0.8215931203314293

In [144]:
pred = pipe.predict(test_scaled)#.sum()

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.9s remaining:    2.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.7s finished


In [169]:
ID

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [171]:
len(pred)

418

In [145]:
pd.DataFrame({'PassengerId':ID, 'Survived':pred}).to_csv('submission_-6.csv', index=False)

In [204]:
pred.sum()

143

In [3]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [200]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [132]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier

In [197]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier#, XGBRFClassifer
from sklearn.preprocessing import OrdinalEncoder


In [217]:
dd = BaggingClassifier(base_estimator=XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123), oob_score=True, verbose=True, n_estimators=100, max_samples=0.8, bootstrap=True, n_jobs=-1)

In [233]:
numeric_features = ['Age', 'Fare']
categorical_features = ['Embarked', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Cabin']


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer( strategy='mean'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
   # ('enc', OrdinalEncoder( handle_unknown='error')),
    ('onehot', OneHotEncoder( handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor), 
                      ('svd', TruncatedSVD()),
                      
                      ('classifier', XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123) )])

X_train, X_test, y_train, y_test = train_test_split(train.drop(['Survived',  'Name','Ticket'], axis=1),train['Survived'] , test_size=0.2, random_state=0)

#clf.fit(X_train, y_train)

In [242]:
# Create arrays for the features and the target: X, y
#X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]
import xgboost as xgb 
# Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X_train, label=y_train, enable_categorical=True)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":3}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=3, num_boost_round=5, 
                  metrics="error", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the accuracy
print(((1-cv_results["test-error-mean"]).iloc[-1]))

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.Sex, Cabin, Embarked

In [234]:
svc = SVC()

In [235]:
parameters = {'C':[0.1, 1, 10], 'gamma':[0.00001, 0.0001, 0.001, 0.01, 0.1]}
searcher = Pipeline([('prepro', preprocessor), ('search',GridSearchCV(svc, parameters) )]) 

In [236]:
searcher.fit(X_train, y_train)

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex', 'Pclass',
                                                   'SibSp', 'Parch',
                                                   'Cabin'])])),
                ('search',
           

In [237]:
searcher[1].best_score_

0.7795331429134246

In [238]:
clf.fit(X_train, y_train)

[05:57:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex', 'Pclass',
                                                   'SibSp', 'Parch',
                                                   'Cabin'])])),
                ('svd', Truncate

In [239]:
clf.score(X_test, y_test)

0.7206703910614525

In [240]:
clf.score(X_train, y_train)

0.8132022471910112

In [ ]:
pred = clf.predict(X_test)

In [ ]:
parameters = {'classifier__'}

In [ ]:
clf_gs = GridSearchCV(clf, param_grid=parameters)

In [258]:
clf.fit(train.drop(['Survived', 'Name','Ticket'], axis=1), train['Survived'])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   10.2s remaining:   30.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   12.0s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('enc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex',

In [261]:
clf.score(X_test, y_test)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.8s remaining:    2.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.4s finished


0.9553072625698324

In [260]:
clf.score(train.drop(['Survived', 'Name','Ticket'], axis=1), train['Survived'])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.8s remaining:    2.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.4s finished


0.9371492704826038

In [248]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [249]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [262]:
pred = clf.predict(test.drop(['Name', 'Ticket'], axis=1))

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.8s remaining:    2.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.4s finished


In [263]:
pd.DataFrame({'PassengerId':ID, 'Survived':pred}).to_csv('submission_-7.csv', index=False)